In [1]:
import ee
%matplotlib inline
import math
import warnings
import intake
warnings.filterwarnings('ignore')
from collections import defaultdict
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats

import xarray as xr
xr.set_options(display_style='html')

import datetime, calendar

plt.style.use("seaborn-darkgrid")

#ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1Adeu5BWfWmJRR8ZXwwzjPonfJRMcQ9iRlYJTTKNQIHGLFrg1K_laKbAhONE

Successfully saved authorization token.


In [2]:
HIST_START = 1980
HIST_END = 2014
FUTURE_START = 2080
FUTURE_END = 2099

PERCENTILE_STARTYEAR = 1980
PERCENTILE_ENDYEAR = 2019

NUM_BEST_MODELS = 3

In [3]:
MODELS = {
    'tasmax': ('CanESM5', 'MRI-ESM2-0', 'CAMS-CSM1-0'),
    'tasmin': ('CanESM5', 'MRI-ESM2-0'),
    'pr': ('CanESM5', 'MRI-ESM2-0', 'CAMS-CSM1-0' ),
    'hurs': ('CanESM5', 'MRI-ESM2-0', 'UKESM1-0-LL')
}
RUNS = {
    'CanESM5': 'r1i1p1f1',
    'MRI-ESM2-0': 'r1i1p1f1',
    'CAMS-CSM1-0': 'r2i1p1f1',
    #'UKESM1-0-LL': 'r1i1p1f2'  # Uses 360 day
}

YEARLENGTH = {
    'ERA5': 366,
    'CanESM5': 365,
    'MRI-ESM2-0': 366,
    'CAMS-CSM1-0': 366
}

In [4]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)

In [5]:
VARIABLES = {
    'tasmax': {
        'era_varname': 'maximum_2m_air_temperature',
        'nex_transform': lambda x: x - 273.5,
        'era_transform': lambda x: x - 273.5
    },
    'tasmin': {
        'era_varname': 'minimum_2m_air_temperature',
        'nex_transform': lambda x: x - 273.5,
        'era_transform': lambda x: x - 273.5
    },
    'pr': {
        'era_varname': 'total_precipitation',
        'nex_transform': lambda x: x * 86400,
        'era_transform': lambda x: x * 1000
    }   
}

In [6]:
def calendardate_percentiles(nex_varname, q, latlon, sh_hem=False):
    era_varname = VARIABLES[nex_varname]['era_varname']
    hist_start = PERCENTILE_STARTYEAR
    hist_end = PERCENTILE_ENDYEAR
    allyears = []
    for year in range(hist_start, hist_end):
        allyears.append(VARIABLES[nex_varname]['era_transform'](get_var(era_varname, 'ERA5', latlon, start_year=year, end_year=year, southern_hem=False)))
    if not sh_hem:
        return np.percentile(np.vstack(allyears), q, axis=0)
    else:
        res = np.percentile(np.vstack(allyears), q, axis=0)
        return np.concatenate([res[152:], res[:152]])

def wholeyear_percentile(nex_varname, q, latlon):
    era_varname = VARIABLES[nex_varname]['era_varname']
    hist_start = PERCENTILE_STARTYEAR
    hist_end = PERCENTILE_ENDYEAR
    allyears = []
    for year in range(hist_start, hist_end):
        allyears.append(VARIABLES[nex_varname]['era_transform'](get_var(era_varname, 'ERA5', latlon, start_year=year, end_year=year, southern_hem=False)))
    return np.percentile(np.concatenate(allyears).flatten(), q)

def yearextreme_percentile(nex_varname, q, latlon, wantmax):
    era_varname = VARIABLES[nex_varname]['era_varname']
    hist_start = PERCENTILE_STARTYEAR
    hist_end = PERCENTILE_ENDYEAR
    allyears = []
    for year in range(hist_start, hist_end):
        allyears.append([np.min, np.max][int(wantmax)](VARIABLES[nex_varname]['era_transform'](get_var(era_varname, 'ERA5', latlon, start_year=year, end_year=year, southern_hem=False))))
    return np.percentile(np.array(allyears), q)

def d2j(datestring):
    d = datetime.date.fromisoformat(datestring)
    jday = d.timetuple().tm_yday
    if calendar.isleap(d.year) and jday > 59:
        jday -= 1
    return jday

def get_rmsd(d1, d2):
    c1 = seasonal_means(d1)
    c2 = seasonal_means(d2)
    return np.sqrt(np.mean(np.sum((c1 - c2)**2)))

def count_runs(tf_array, min_runsize):
    falses = np.zeros(tf_array.shape[0]).reshape((tf_array.shape[0],1))
    extended_a = np.concatenate([[0], tf_array, [0]])
    df = np.diff(extended_a)
    starts = np.nonzero(df == 1)[0]
    ends = np.nonzero(df == -1)[0]
    count = 0
    for idx in range(starts.size):
        if ends[idx] - starts[idx] >= min_runsize:
            count += 1
    return count



def get_var(varname, model, latlon, start_year, end_year, southern_hem=False, extralong=False, scenario='ssp585'):
    def removeLeapDays(arr, extralong_=False, southern_hem_=False):
        if extralong_:
            indices = list(range(184))
            jan1_idx = 184
            
            for year in range(start_year, end_year+1):
                indices += [jan1_idx + i for i in range(365)]
                jan1_idx += 365
                if calendar.isleap(year):
                    jan1_idx += 1
            return arr[indices]
        elif not southern_hem_:
            indices = []
            jan1_idx = 0
            for year in range(start_year, end_year+1):
                indices += [jan1_idx + i for i in range(365)]
                jan1_idx += 365
                if calendar.isleap(year):
                    jan1_idx += 1
            return arr[indices]
        else:
            indices = []
            jul1_idx = 0
            for year in range(start_year-1, end_year):
                indices += [jul1_idx + i for i in range(365)]
                jul1_idx += 365
                if calendar.isleap(year):
                    jul1_idx += 1
            return arr[indices]
    
    if model != 'ERA5' and start_year < 2015 and end_year >= 2015:
        raise Exception("Requesting hist and non-hist variables in one query")
    if model == 'ERA5':
        dataset = ee.ImageCollection("ECMWF/ERA5/DAILY")
        gee_geom = ee.Geometry.Point((latlon[1], latlon[0]))
        if extralong:
            data_vars = dataset.select(varname).filter(ee.Filter.date('{0}-07-01'.format(start_year-1), '{0}-01-01'.format(end_year+1)))
            result = [i[4] for i in data_vars.getRegion(gee_geom, 2500, 'epsg:4326').getInfo()[1:]]
            return removeLeapDays(np.array(result), extralong_=True, southern_hem_=False)
        elif not southern_hem:
            data_vars = dataset.select(varname).filter(ee.Filter.date('{0}-01-01'.format(start_year), '{0}-01-01'.format(end_year+1)))
            result = [i[4] for i in data_vars.getRegion(gee_geom, 2500, 'epsg:4326').getInfo()[1:]]
            return removeLeapDays(np.array(result), extralong_=False, southern_hem_=False)
        else:
            data_vars = dataset.select(varname).filter(ee.Filter.date('{0}-07-01'.format(start_year-1), '{0}-07-01'.format(end_year)))
            result = [i[4] for i in data_vars.getRegion(gee_geom, 2500, 'epsg:4326').getInfo()[1:]]
            return removeLeapDays(np.array(result), extralong_=False, southern_hem_=True)
    else:  # Retrieve NetCDF from pangeo archive
        
    # NOTE: IT WOULD BE BETTER TO GET DATA FROM AWS, LIKE THIS:
        
    #    def s3open(path):
    #        fs = s3fs.S3FileSystem(anon=True, default_fill_cache=False, 
    #                               config_kwargs = {'max_pool_connections': 50})
    #        return s3fs.S3Map(path, s3=fs)
    #    testfile = s3open('s3://cmip6-pds/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-ESM4/ssp119/r1i1p1f1/day/tasmax/gr1/v20180701/')
    #    # (Get path from data catalog csv)
    #    ds = xr.open_mfdataset([testfile], engine='zarr', parallel=True)
    #    a = ds['tasmax'].sel(time=slice('2030-01-01', '2039-12-31')).sel(lat=lat, lon=lon, method='nearest').to_numpy()
        
    # BUT THIS LAST STEP TAKES A VERY LONG TIME
        
        scenario = [scenario, 'historical'][int(start_year < 2015)]
        cat = col.search(experiment_id=scenario, table_id='day', variable_id=varname, source_id=model, member_id=RUNS[model], grid_label='gn')
        ds_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True})
        if extralong:
            ds = list(ds_dict.values())[0][varname][0][0].sel(time=slice('{0}-07-01'.format(start_year-1), '{0}-12-31'.format(end_year))).sel(lat=latlon[0], lon=latlon[1], method='nearest').to_numpy()
            if YEARLENGTH[model] != 365:
                return removeLeapDays(np.array(ds), extralong_=True)
            else:
                return np.array(ds)
        elif not southern_hem:
            ds = list(ds_dict.values())[0][varname][0][0].sel(time=slice('{0}-01-01'.format(start_year), '{0}-12-31'.format(end_year))).sel(lat=latlon[0], lon=latlon[1], method='nearest').to_numpy()
            if YEARLENGTH[model] != 365:
                return removeLeapDays(np.array(ds), extralong_=False, southern_hem_=False)
            else:
                return np.array(ds)
        else:
            ds = list(ds_dict.values())[0][varname][0][0].sel(time=slice('{0}-07-01'.format(start_year-1), '{0}-06-30'.format(end_year))).sel(lat=latlon[0], lon=latlon[1], method='nearest').to_numpy()
            if YEARLENGTH[model] != 365:
                return removeLeapDays(np.array(ds), southern_hem_=True)
            else:
                return np.array(ds)

    
def quarters(d, start_year, end_year, southern_hem=False):
    q2 = []  # 60-151
    q3 = []  # 152-243
    q4 = []  # 244-334
    q1 = []  # 335-59
    if not southern_hem:
        jan1_idx = 365
        for year in range(start_year, end_year):
            tmp = np.concatenate((d[jan1_idx - 365 : jan1_idx - 365 + 60], d[jan1_idx + 335 : jan1_idx + 365]), axis=0)
            q1.append(tmp)
            q2.append(d[jan1_idx + 60 : jan1_idx + 152])
            q3.append(d[jan1_idx + 152 : jan1_idx + 244])
            q4.append(d[jan1_idx + 244 : jan1_idx + 335])

            jan1_idx += 365 + [0, 0][int(False and calendar.isleap(year))]
        mam_res = np.vstack(q2)
        jja_res = np.vstack(q3)
        son_res = np.vstack(q4)
        djf_res = np.vstack(q1)
    else:
        jul1_idx = 365
        for year in range(start_year, end_year):
            tmp = np.concatenate((d[jul1_idx - 365 : jul1_idx - 365 + 60], d[jul1_idx + 335 : jul1_idx + 365]), axis=0)
            q3.append(tmp)
            q4.append(d[jul1_idx + 60 : jul1_idx + 152])
            q1.append(d[jul1_idx + 152 : jul1_idx + 244])
            q2.append(d[jul1_idx + 244 : jul1_idx + 335])

            jul1_idx += 365 + [0, 0][int(False and calendar.isleap(year))]
        mam_res = np.vstack(q4)
        jja_res = np.vstack(q1)
        son_res = np.vstack(q2)
        djf_res = np.vstack(q3)
    return mam_res, jja_res, son_res, djf_res
    
def seasonal_means(d):
    q = quarters(d, HIST_START, HIST_END)
    return np.array([np.mean(q[0], axis=1), np.mean(q[1], axis=1), np.mean(q[2], axis=1), np.mean(q[3], axis=1)])

def calibration_function(hist_obs, hist_mod):
# Calibration functions are P-P plots of historical and modeled values

    source = np.sort(hist_obs.flatten())
    target= np.sort(hist_mod.flatten())
   
    if (np.max(source) == 0 and np.min(source) == 0):
        return np.arange(0, target.size) / target.size
    if (np.max(target) == 0 and np.min(target) == 0):
        return np.arange(0, source.size) / source.size
    new_indices = []

    for target_idx, target_value in enumerate(target):
        if target_idx < len(source):
            source_value = source[target_idx]
            if source_value > target[-1]:
                new_indices.append(target.size - 1)
            else:
                new_indices.append(np.argmax(target >= source_value))
    return np.array(new_indices) / source.size

def calibrate_component(uncalibrated_data, calibration_fxn):
    N = len(uncalibrated_data)
    unsorted_uncalib = [(i, idx) for idx, i in enumerate(uncalibrated_data)]
    sorted_uncalib = sorted(unsorted_uncalib)
    result = [0] * N
    for j in range(N):
        X_j = j / (N + 1)
        Y_jprime = calibration_fxn[math.floor(X_j * len(calibration_fxn))]
        jprime = math.floor(Y_jprime * (N + 1))
        result[sorted_uncalib[j][1]] = sorted_uncalib[min(len(sorted_uncalib)-1, jprime)][0]
    
    return result

def calibrate(uncalibrated_data, calibration_fxn):
    mam = []
    jja = []
    son = []
    djf = []
    mam_idx = []
    jja_idx = []
    son_idx = []
    djf_idx = []
    for idx, i in enumerate(uncalibrated_data):
        if idx % 365 >= 60 and idx % 365 < 152:
            mam.append(uncalibrated_data[idx])
            mam_idx.append(idx)
        elif idx % 365 >= 152 and idx % 365 < 244:
            jja.append(uncalibrated_data[idx])
            jja_idx.append(idx)
        elif idx % 365 >= 244 and idx % 365 < 335:
            son.append(uncalibrated_data[idx])
            son_idx.append(idx)
        else:
            djf.append(uncalibrated_data[idx])
            djf_idx.append(idx)
    
    mam_calib = calibrate_component(np.array(mam), calibration_fxn[0])
    jja_calib = calibrate_component(np.array(jja), calibration_fxn[1])
    son_calib = calibrate_component(np.array(son), calibration_fxn[2])
    djf_calib = calibrate_component(np.array(djf), calibration_fxn[3])
    
    result = [0] * len(uncalibrated_data)
    for i in range(len(mam_idx)):
        result[mam_idx[i]] = mam_calib[i]
    for i in range(len(jja_idx)):
        result[jja_idx[i]] = jja_calib[i]
    for i in range(len(son_idx)):
        result[son_idx[i]] = son_calib[i]
    for i in range(len(djf_idx)):
        result[djf_idx[i]] = djf_calib[i]

    return np.array(result)

def get_gamma(count, size):
    return np.random.gamma(shape = count + 0.5, size=size)
def get_beta(count, num, size):
    return np.random.beta(a = count + 0.5, b = num - count + 0.5, size=size)

In [17]:
class Hazard:
    pass

class Tempwave_simple(Hazard):
    def __init__(self, varname, min_duration, threshold, want_gte=True):
        if type(threshold) == np.ndarray and threshold.size % 365 != 0:
            raise Exception('Comparison array length is not an integer multiple of 365')
        self.varname = varname
        self.want_gte = want_gte
        self.min_duration = min_duration
        self.threshold = threshold  # May be scalar or 365-long array
        self.probmodel = 'Poisson'
    def count(self, datalist):
        data = datalist[0]
        if type(self.threshold) in (float, int, np.float64, np.int32):
            threshold = self.threshold
        else:   # type is np array
            threshold = np.array([])
            while threshold.size < data.size:
                threshold = np.concatenate([threshold, self.threshold])
        if self.want_gte:
            tf_array = data >= threshold
        else:
            tf_array = data <= threshold
        return count_runs(tf_array, self.min_duration)
    
class Heatwave_highlow(Hazard):
    def __init__(self, hightemp, lowtemp, min_duration):
        self.varname = 'tasmax+tasmin'
        self.min_duration = min_duration
        self.hightemp = hightemp
        self.lowtemp = lowtemp
        self.probmodel = 'Poisson'
    def count(self, datalist):
        data_tx = datalist[0]
        data_tn = datalist[1]
        if type(self.hightemp) in (float, int, np.float64, np.int32):
            high_threshold = self.hightemp
        else:   # type is np array
            high_threshold = np.array([])
            while high_threshold.size < data_tx.size:
                high_threshold = np.concatenate([high_threshold, self.hightemp])
        if type(self.lowtemp) in (float, int, np.float64, np.int32):
            low_threshold = self.lowtemp
        else:   # type is np array
            low_threshold = np.array([])
            while low_threshold.size < data_tn.size:
                low_threshold = np.concatenate([low_threshold, self.lowtemp])
        tf_array_tx = data_tx >= high_threshold
        tf_array_tn = data_tn >= low_threshold
        return count_runs(tf_array_tx * tf_array_tn, self.min_duration)

class Threshold_simple(Hazard):
    def __init__(self, varname, threshold, want_gte):
        self.varname = varname
        self.threshold = threshold
        self.want_gte = want_gte
        self.probmodel = 'binomial'
    def count(self, datalist):
        data = datalist[0]
        if data.size % 365 != 0:
            raise Exception('Data array length is not an integer multiple of 365')
        byyear = data.reshape(data.size//365, 365)
        if self.want_gte:
            return np.sum((np.max(byyear, axis=1) >= self.threshold) * 1)
        else:
            return np.sum((np.max(byyear, axis=1) <= self.threshold) * 1)

class Hotdays_inrange(Hazard):
    def __init__(self, hightemp, lowtemp):
        self.varname = 'tasmax'
        self.hightemp = hightemp
        self.lowtemp = lowtemp
        self.probmodel = 'binomial'
    def count(self, datalist):
        data = datalist[0]
        if data.size % 365 != 0:
            raise Exception('Data array length is not an integer multiple of 365')
        tf_array_high = data <= self.hightemp
        tf_array_low = data >= self.lowtemp
        return runs(tf_array_high * tf_array_low, self.min_duration, 'count')

In [8]:
class Location:
    def __init__(self, name, latlon):
        self.name = name
        self.latlon = latlon
        self.hist_observed = {}
        self.hist_modeled = {}
        self.best_models = {}
        self.fut_modeled = {}
        self.calib_fxns = {}
        
    def get_data(self, varname):
    # Gets historical observations from ERA5 Daily Aggregate (from GEE)
    # Goes through all NEX-GDDP-CMIP6 models in GEE and gets historical model outputs
    # Chooses three best models based on quarterly RMSD
    
        hist_obs = VARIABLES[varname]['era_transform'](get_var(VARIABLES[varname]['era_varname'], 'ERA5', self.latlon, HIST_START, HIST_END, southern_hem=False))
        hist_mods = {}
        rmsds = []
        for model in MODELS[varname]:
            hist_mod = VARIABLES[varname]['nex_transform'](get_var(varname, model, self.latlon, HIST_START, HIST_END, southern_hem=False))
            hist_mods[model] = hist_mod
            rmsds.append((get_rmsd(hist_obs, hist_mod), model))
        rmsds.sort()
        
        '''best_models = []
        families = []
        idx = 0
        while len(best_models) < 3:
            if not MODEL_INFO[rmsds[idx][1]] in families:
                best_models.append(rmsds[idx][1])
                families.append(MODEL_INFO[rmsds[idx][1]])
            idx += 1

        for m in best_models:
            print(m, [i[0] for i in rmsds if i[1]==m][0])'''
        best_models = []
        for idx in range(min(NUM_BEST_MODELS, len(MODELS[varname]))):
            best_models.append(rmsds[idx][1])
            
        self.hist_observed[varname] = hist_obs
        self.hist_modeled[varname] = hist_mods
        self.best_models[varname] = best_models
        
    # Get calibration functions
        self.calib_fxns[varname] = {}
        hist_obs = self.hist_observed[varname]
        hist_mod = self.hist_modeled[varname]
        for model in self.best_models[varname]:
            o_quarters = quarters(hist_obs, HIST_START, HIST_END)
            m_quarters = quarters(hist_mod[model], HIST_START, HIST_END)
            self.calib_fxns[varname][model] = [calibration_function(o_quarters[i].flatten(), m_quarters[i].flatten()) for i in range(4)]
            
    # Get future model outputs
        self.fut_modeled[varname] = {
            model: VARIABLES[varname]['nex_transform'](get_var(varname, model, self.latlon, FUTURE_START, FUTURE_END, extralong=True)) for model in best_models
        }
        
            

In [9]:
class Estimate:
    def __init__(self, location, hazard, future_start, future_end, sh_year):
        self.location = location
        self.hazard = hazard
        self.future_start = future_start
        self.future_end = future_end
        self.sh_year = sh_year 
        self.estimate = {}
        
        for varname in self.hazard.varname.split('+'):
            if not varname in list(self.location.hist_observed.keys()):
                self.location.get_data(varname)
        
    def future_count(self):
    # Gets future model outputs for three best models
    # Calibrates based on stored model-specific quarterly calibration functions
    # Calculates event count within future time series
    # Draws 10,000 posterior rate parameters from appropriate Jeffrys prior distribution (parameterized with count)
    # For each rate parameter, draw one event count
    
        fut_mod = {}
        varnames = self.hazard.varname.split('+')
        for varname in varnames:
            fut_mod[varname] = {}
            for model in self.location.best_models[varname]:
                if self.sh_year:
                    fut_mod[varname][model] = self.location.fut_modeled[varname][model][:-184]
                else:
                    fut_mod[varname][model] = self.location.fut_modeled[varname][model][184:]
        best_models = []
        for idx in range(min(len(MODELS[varname]), NUM_BEST_MODELS)):
            best_models.append('+'.join([self.location.best_models[varname][idx] for varname in varnames]))
        posterior_rateparams = {}
        posterior_draws = {}
        estimate = {}
        for modelplus in best_models:
            calib_data = []
            for idx, varname in enumerate(varnames):
                model = modelplus.split('+')[idx]
                calib_data.append(np.array(calibrate(fut_mod[varname][model], self.location.calib_fxns[varname][model])))
            if self.sh_year:
                count = self.hazard.count(calib_data)
            else:
                count = self.hazard.count([cd[152:-213] for cd in calib_data])
            if self.hazard.probmodel == 'Poisson':
                posterior_rateparams[modelplus] = get_gamma(count, 10000)
                posterior_draws[modelplus] = np.random.poisson(posterior_rateparams[modelplus], 10000)
            else:  # self.hazard.probmodel == 'binomial'
                posterior_rateparams[modelplus] = get_beta(count, self.future_end - self.future_start + 1, 10000)
                posterior_draws[modelplus] = np.random.binomial(self.future_end - self.future_start + 1, posterior_rateparams[modelplus], 10000)
            self.estimate[modelplus] = np.sum(posterior_draws[modelplus]) / 10000
        
        print()
        for modelplus in best_models:
            if self.hazard.probmodel == 'Poisson':
                print('{0}: {1}'.format(modelplus, self.estimate[modelplus]))
            else:
                print('{0}: {1}'.format(modelplus, '{0:.1f}%'.format(self.estimate[modelplus] / (self.future_end - self.future_start + 1) * 100)))

In [10]:
future_years = [(2030, 2049), (2080, 2099)]

In [ ]:
with open('citiesonly.csv', 'r', encoding='utf-8') as ifile:
    locations = {}
    for line in ifile.readlines()[1:]:
        items = line.split(',')
        locations[items[0]] = (float(items[1]), float(items[2]))
for location_name in locations:
    print(location_name)
    latlon = locations[location_name]
    
    high_95c = calendardate_percentiles('tasmax', 95, latlon, sh_hem=latlon[0] < 0)
    cold_5n = wholeyear_percentile('tasmin', 5, latlon)
    test_hazards = [
        {'name': 'Heat wave 95th percentile', 'obj': Tempwave_simple('tasmax', 5, high_95c, True), 'sh_year': latlon[0] < 0},
        {'name': 'Days warmer than 35C', 'obj': Threshold_simple('tasmax', 35, want_gte=True), 'sh_year': latlon[0] < 0},
        {'name': 'Days colder than than 5th pctle yearlong', 'obj': Threshold_simple('tasmin', cold_5n, want_gte=False), 'sh_year': False},
        {'name': 'Days rainier than than 500mm', 'obj': Threshold_simple('pr', 500, want_gte=True), 'sh_year': False},
    ]
    print('  95th pctl maxtemp = {0:.1f}'.format(np.max(high_95c)))
    print('  5th pctl mintemp = {0:.1f}'.format(cold_5n))
    loc = Location(location_name, latlon)
    with open('test_outputs_1.csv', 'w') as ofile:
        ofile.write('City,Hazard,')
        for year_range in future_years:
            for prefix in ('low', 'mid', 'high'):
                ofile.write('{0}_{1}-{2},'.format(prefix, year_range[0], year_range[1]))
        ofile.write('\n')
        for haz in test_hazards:
            ofile.write('{0},{1}'.format(loc.name, haz['name']))
            for fut_start, fut_end in future_years:
                print(haz['name'])
                print(fut_start, fut_end)
                est = Estimate(loc, haz['obj'], fut_start, fut_end, haz['sh_year'])
                est.future_count()
                res = list(est.estimate.values())
                res.sort()
                if haz['obj'].probmodel == 'binomial':
                    res = ['{0:.1f}%'.format(i / (fut_end - fut_start + 1) * 100) for i in res]
                else:
                    res = ['{0:.1f}'.format(i / (fut_end - fut_start + 1)) for i in res]
                ofile.write(',{0}'.format(res[0]))
                if len(res) == 3:
                    ofile.write(',{0}'.format(res[1]))
                else:
                    ofile.write(',')
                ofile.write(',{0}'.format(res[-1]))
            ofile.write('\n')

Dubai AE


In [34]:
for quarter in range(4):
    obs_10 = np.percentile(quarters(hist_obs_tx, HIST_START, HIST_END)[quarter], 10)
    obs_90 = np.percentile(quarters(hist_obs_tx, HIST_START, HIST_END)[quarter], 90)
    for model in best_models_tx:
        mod = quarters(hist_mods_tx[model] - 273.15, HIST_START, HIST_END)[quarter].flatten()
        print('{0}: min modeled value does not exceed observed 10th percentile  {1}'.format(model, min(mod) <= obs_10))
        print('{0}: max modeled value does not exceed observed 90th percentile  {1}'.format(model, max(mod) >= obs_90))


GFDL-CM4: min modeled value does not exceed observed 10th percentile  True
GFDL-CM4: max modeled value does not exceed observed 90th percentile  True
CanESM5: min modeled value does not exceed observed 10th percentile  True
CanESM5: max modeled value does not exceed observed 90th percentile  True
ACCESS-CM2: min modeled value does not exceed observed 10th percentile  True
ACCESS-CM2: max modeled value does not exceed observed 90th percentile  True
GFDL-CM4: min modeled value does not exceed observed 10th percentile  True
GFDL-CM4: max modeled value does not exceed observed 90th percentile  True
CanESM5: min modeled value does not exceed observed 10th percentile  True
CanESM5: max modeled value does not exceed observed 90th percentile  True
ACCESS-CM2: min modeled value does not exceed observed 10th percentile  True
ACCESS-CM2: max modeled value does not exceed observed 90th percentile  True
GFDL-CM4: min modeled value does not exceed observed 10th percentile  True
GFDL-CM4: max modeled